# Data Validation Tutorial
Welcome to the data validation tutorial using tensorflow data validation! In this tutorial we will be using tensorflow data validation (or tfdv) to write data tests. The tutorial is divided into the following segments:

1. TFDV Exploration: we will start by exploring the fundamentals of tfdv.
2. Data validation: next we will write some *tests* for our data.

In [2]:
# put your import statements here
import tensorflow_data_validation as tfdv
import pandas as pd
from sklearn.model_selection import train_test_split


2022-06-18 16:51:05.360787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-18 16:51:05.360810: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Part 1: TFDV Exploration
In this section we will explore the basics of tfdv. Follow the instructions below.

### 1.1 Load the data
Use `pandas` to load the csv file as a dataframe.

+ *Hint*: did you check the [10 minutes with pandas guide]? Did you read the *Getting data in/out* section?
+ *Hint*: did you import pandas as `pd`?

[10 minutes with pandas guide]: https://pandas.pydata.org/docs/user_guide/10min.html

In [4]:
train_df = pd.read_csv(f'../data/train.tsv', sep="\t")
val_df = pd.read_csv(f'../data/validation.tsv', sep="\t")
test_df = pd.read_csv(f'../data/test.tsv', sep="\t")


### 1.3 Use tfdv to compute statistics for the training set
Generate descriptive statistics for the training set using tfdv.

+ *Hint*: you already know what to do...[read the docs]! Look for the appropriate `generate_statistics_from...` method.
+ *Hint*: did you import tensorflow data validation as `tfdv`?

[read the docs]: https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv

In [5]:
train_stats = tfdv.generate_statistics_from_dataframe(train_df)
val_stats = tfdv.generate_statistics_from_dataframe(val_df)
schema = tfdv.infer_schema(train_stats)


### 1.4 Visualise training stats & make observations
Lets visualise the statistics for our training set & see how it looks like. In particular, make the following observations:

1. Look at the distribution of the features
1. Look at the range of values for numerical features
1. Look at the values for categorical features
1. Do we have any missing values?
1. Explore the UI: change the feature sort order, change the chart type, etc.

+ *Hint*: there is only one method in tfdv that can do this!

In [6]:
# your code here
tfdv.visualize_statistics(train_stats)

***Reflect on the following questions:***

1. Is the visualisation provided by tfdv useful?
1. Can tfdv be used during data exploration & understanding?
1. Do we have any features that do not follow a *normal distribution*? Will this affect the model's performance?

### 1.5 Visualise testing stats & make observations
Lets now also make sure that our test set is *similar* to our training set. Generate the statistics for the test set & visulise it alongside the train set.

+ *Hint*: did you read the api documentation for the method you used to visualise statistics carefully? Can you visualise statistics of two datasets at the same time?

In [7]:
# your code here

tfdv.visualize_statistics(rhs_statistics=train_stats, lhs_statistics=val_stats, rhs_name="TRAIN", lhs_name="VALIDATION")

In [8]:

train_df = train_df.drop(columns=['tags'])

train_stats = tfdv.generate_statistics_from_dataframe(train_df)
test_stats = tfdv.generate_statistics_from_dataframe(test_df)
tfdv.visualize_statistics(rhs_statistics=train_stats, lhs_statistics=test_stats, rhs_name="TRAIN", lhs_name="TEST")

***Reflect on the following questions:***

1. Are the training & testing splits similar?
1. Can tfdv be used during data exploration & understanding?

## Part 2: Data Validation
In this section we will write tests for our data. TFDV does this by first creating a *schema* for our data which specifies what we expect our data to look like. In SE terms, the schema is the *test oracle* which we can use to check if our tests are passing or failing.

### 2.1 Create a schema
Generate a schema from the training set.

In [ ]:
# your code here
schema = tfdv.infer_schema(train_stats)

### 2.2 Inspect the schema
"Pretty print" the schema that was generated.

In [ ]:
# your code here
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'id',INT,required,,-
'diagnosis',STRING,required,,'diagnosis'
'radius_mean',FLOAT,required,,-
'texture_mean',FLOAT,required,,-
'perimeter_mean',FLOAT,required,,-
'area_mean',FLOAT,required,,-
'smoothness_mean',FLOAT,required,,-
'compactness_mean',FLOAT,required,,-
'concavity_mean',FLOAT,required,,-


,Values
Domain,
'diagnosis',"'B', 'M'"


***Reflect on the following:***

+ Does the schema make sense?
+ Does it need to be modified?
+ What about this `Unnamed: 32` column?
+ Do we really need the `id` column?

#### 2.2.1 Cleanup the schema
Drop the columns we don't need & recreate the schema.

In [ ]:
# your code here
train = train.drop(columns=['id', 'Unnamed: 32'])
train.shape

(426, 31)

### 2.3 Inspect anomalies in test set
Now lets check if our test set meets the expectations that we define in our schema. First, use tfdv to find the *anomalies* (a.k.a bugs) in our test set and then "pretty print" them.

+ *Hint*: I will give you the answer for this one! You need to use the `validate_statistics` method followed by the appropriate `display...` method.

In [ ]:
train_stats = tfdv.generate_statistics_from_dataframe(train)

In [ ]:
schema = tfdv.infer_schema(train_stats)

In [ ]:
# your code here
anomalies = tfdv.validate_statistics(test_stats, schema=schema)

In [ ]:
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Unnamed: 32',New column,New column (column in data but not in schema)
'id',New column,New column (column in data but not in schema)


### (OPTIONAL) 2.4 Make the bugs go away!
Can you fix the issue here? We did this for the training set already!

In [ ]:
# your code here
test = test.drop(columns=['Unnamed: 32', 'id'])
test.shape

(143, 31)

In [ ]:
test_stats = tfdv.generate_statistics_from_dataframe(test)
anomalies = tfdv.validate_statistics(test_stats, schema=schema)
tfdv.display_anomalies(anomalies)